In [1]:
import tensorflow as tf
import numpy as np

char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
           'h', 'i', 'j', 'k', 'l', 'm', 'n',
           'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z']

# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k': 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']



# 1. 입력용으로, 단어의 처음 세글자의 알파벳 인덱스를 구한 배열을 만듭니다.
#    - input = [num_dic[n] for n in seq[:-1]]
# 2. 출력값용으로, 마지막 글자의 알파벳 인덱스를 구합니다.
#    - target = num_dic[seq[-1]]
# 3. 입력인코딩으로 변환합니다.
#    - input_batch.append(np.eye(dic_len)[input])
# 4. 실측값은 원-핫 인코딩하지 않고 그대로 사용할 것입니다. (ex. 15 )
#    - target_batch.append([target])
#    손실함수로 sparse_softmax_cross_entropy_with_logits를 사용할 것이기 때문입니다.
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch


/usr/local/Cellar/python3/3.6.4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

# 단어의 전체 중 처음 3글자를 단계적으로 학습할 것이므로 n_step은 3이 됩니다.
# 입력값과 출력값은 알파벳의 원-핫 인코딩을 사용할 것이므로 알파벳 글자들의 배열 크기인 dic_len과 같습니다.
# sparse_softmax_cross_entropy_with_logits함수를 사용하더라도 바교를 위한 예측 모델의 출력값은 원-핫 인코딩을 사용해야 합니다.
# 그래서 n_class값도 n_input값과 마찬가지로 dic_len과 크기가 같도록 설정했습니다.
n_step = 3
n_input = n_class = dic_len

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)



In [3]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    
print('최적화 완료!')

prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)



Epoch: 0001 cost = 3.694737
Epoch: 0002 cost = 2.951665
Epoch: 0003 cost = 1.799197
Epoch: 0004 cost = 1.272629
Epoch: 0005 cost = 0.722166
Epoch: 0006 cost = 0.501834
Epoch: 0007 cost = 0.752575
Epoch: 0008 cost = 0.497234
Epoch: 0009 cost = 0.280986
Epoch: 0010 cost = 0.198005
Epoch: 0011 cost = 0.276617
Epoch: 0012 cost = 0.513078
Epoch: 0013 cost = 0.173800
Epoch: 0014 cost = 0.150556
Epoch: 0015 cost = 0.227897
Epoch: 0016 cost = 0.139296
Epoch: 0017 cost = 0.115864
Epoch: 0018 cost = 0.158315
Epoch: 0019 cost = 0.175533
Epoch: 0020 cost = 0.052923
Epoch: 0021 cost = 0.059435
Epoch: 0022 cost = 0.035020
Epoch: 0023 cost = 0.030994
Epoch: 0024 cost = 0.008454
Epoch: 0025 cost = 0.012075
Epoch: 0026 cost = 0.004239
Epoch: 0027 cost = 0.084884
Epoch: 0028 cost = 0.001781
Epoch: 0029 cost = 0.007933
Epoch: 0030 cost = 0.002188
최적화 완료!

=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool